<a href="https://colab.research.google.com/github/uresha1995/Research-Methodology/blob/main/Assignment_2_Sentiment_Analysis1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install the libraries

!pip install --upgrade transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [2]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import torch
import re
import random

In [3]:
#Download the dataset

dataset = load_dataset("imdb")
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [4]:
#Print  random reviews

sample_texts = [dataset["train"][i]["text"] for i in random.sample(range(25000), 5)]

for i, text in enumerate(sample_texts):
    print(f"--- Sample {i+1} ---\n{text[:500]}\n")

--- Sample 1 ---
Although the recent re-telling of part of Homer's epic "Troy" with Brad Pitt was entertaining once, "Iphigenia" with the incandescent Irene Pappas is breathtaking. Unfolding in a natural setting with Greek actors speaking their own language lends such authenticity. A chance encounter with this film on one of DirecTV's many movie channels kept me interested in spite of my concentration problems. There is no glitter or "bling" in this movie, just a fabulously rich story impeccably told by actors s

--- Sample 2 ---
A resurrected wrapped monster goes on a murdering binge. A lunatic is seeking revenge against living members of a previous expedition. Universal seems to be running out of wrapping as well as new ideas. Most of the budget was probably spent on Lon Chaney Jr. to star as Kharis, the Mummy.<br /><br />Other players are George Zucco, Wallace Ford, Turhan Bey, Dick Foran and Elyse Knox.<br /><br />How much longer can this madness continue?

--- Sample 3 ---
This is

Preprocessing

In [5]:
#Remove empty and short reviews

def rem_empty(example):
    return len(example["text"].strip()) > 20

dataset["train"] = dataset["train"].filter(rem_empty)
dataset["test"] = dataset["test"].filter(rem_empty)

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [6]:
#Removing unnecessary spaces

def clean_text(example):
    example["text"] = example["text"].strip()
    return example

dataset = dataset.map(clean_text)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
#Remove HTML break tags and replace with space

def clean_text(example):
    text = example["text"]
    text = re.sub(r"<br\s*/?>", " ", text)
    example["text"] = text
    return example

dataset = dataset.map(clean_text)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
#Tokenization

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def preprocess_fun(example):
    return tokenizer(example["text"], truncation=True)

tokenized_datasets = dataset.map(preprocess_fun, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Prepare data for training

In [9]:
#Split dataset into training and test

train_data = tokenized_datasets["train"].shuffle(seed=42).select(range(2000))
test_data = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [10]:
#Loading BERT model

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
!pip uninstall -y transformers

Found existing installation: transformers 4.53.0
Uninstalling transformers-4.53.0:
  Successfully uninstalled transformers-4.53.0


In [1]:
!pip install --quiet "transformers==4.41.0" "datasets==2.18.0" "evaluate==0.4.1"

In [2]:
from transformers import TrainingArguments

training_args = TrainingArguments(

    #Save the model outputs
    output_dir="./results",

    #Run evaluation of each epoch
    evaluation_strategy="epoch",

    #Log training progress every few steps
    logging_strategy="steps",

    #Logging to every 50 steps
    logging_steps=50,

    #Standard fine-tuning learning rate
    learning_rate=2e-5,

    #Tranining batch size
    per_device_train_batch_size=16,

    #Batch size for evaluation
    per_device_eval_batch_size=16,

    #Number of epochs
    num_train_epochs=2,

    # Weight decay for regularization
    weight_decay=0.01,

    #where to store logs
    logging_dir="./logs",

    #loading the best model
    load_best_model_at_end=True,

    #Save the epoch end of the model
    save_strategy="epoch",
    report_to="none"
)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [3]:
from transformers import Trainer, DataCollatorWithPadding

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
)

trainer.train()

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'EncoderDecoderCache' from 'transformers' (/usr/local/lib/python3.11/dist-packages/transformers/__init__.py)